In [3]:
from scipy.ndimage import zoom
import nibabel as nib
import numpy as np
import os
import shutil
from glob import glob   
from tqdm import tqdm

In [19]:
save_path='../../data/registlation_data/'

In [22]:
ct_list=glob('../../data/stroke_ct_nii_256/images/Ischemic/*.nii.gz')
mri_list=[f.replace('_ct','_mri') for f in ct_list]
mri_list=[f.replace('/Ischemic','') for f in mri_list]
for i in tqdm(range(len(mri_list))):
    if os.path.exists(mri_list[i]):
        shutil.copy(ct_list[i],save_path+'CT/'+ct_list[i].split('/')[-1])
        img=nib.load(mri_list[i])
        volume = img.get_fdata()  # (64, 256, 256)
        # z축 축소 비율 계산
        zoom_factor = (40 / 64, 1, 1)

        # trilinear interpolation
        volume_resized = zoom(volume, zoom=zoom_factor, order=1)

        # 저장
        resized_img = nib.Nifti1Image(volume_resized.astype(np.float32), affine=img.affine)
        nib.save(resized_img,save_path+'DWI/'+mri_list[i].split('/')[-1])

  0%|          | 0/1010 [00:00<?, ?it/s]

100%|██████████| 1010/1010 [05:02<00:00,  3.34it/s]


In [29]:
ct_list=glob('../../data/registration_data/CT/*.nii.gz')
mri_list=[f.replace('CT','DWI') for f in ct_list]
for i in tqdm(range(len(mri_list))):
    if os.path.exists(mri_list[i]):
        mri_img=nib.load(mri_list[i])
        mri_volume = mri_img.get_fdata()  
        ct_img=nib.load(ct_list[i])
        ct_volume = ct_img.get_fdata()
        mri_volume=mri_volume+1.
        ct_volume=ct_volume+1.
        mri_img=nib.Nifti1Image(mri_volume.astype(np.float32), affine=mri_img.affine)
        nib.save(mri_img,mri_list[i])
        ct_img=nib.Nifti1Image(ct_volume.astype(np.float32), affine=ct_img.affine)
        nib.save(ct_img,ct_list[i])
        

  0%|          | 0/724 [00:00<?, ?it/s]

100%|██████████| 724/724 [04:10<00:00,  2.89it/s]


In [28]:
ct_img